In [3]:
import pandas as pd

# Load the provided Excel files
attendance_data = pd.read_excel('data/attendance.xlsx')
holiday_data = pd.read_excel('data/holiday.xlsx')
permission_data = pd.read_excel('data/permission.xlsx')

# Display the first few rows of each file to understand their structure
attendance_data.head(), holiday_data.head(), permission_data.head()


(        Date Department Employee  Entry   Exit
 0 2024-07-01         IT    Aynur   9:05  14:50
 1 2024-07-02         IT    Aynur   9:30  18:30
 2 2024-07-03         IT    Aynur  10:10  18:20
 3 2024-07-04         IT    Aynur   9:05  18:12
 4 2024-07-05         IT    Aynur   9:01  18:11,
   Department Employee      Start        End
 0         IT    Aynur 2024-07-10 2024-07-11
 1  Marketinq    Mahir 2024-07-24 2024-08-05
 2  Marketinq    Rasim 2024-08-26 2024-09-03,
         Date Department Employee     Start       End
 0 2024-09-23  Marketinq    Mahir  14:00:00  18:00:00
 1 2024-08-26  Marketinq    Mahir  14:00:00  18:00:00
 2 2024-09-13         IT    Rəsul  15:00:00  18:00:00
 3 2024-09-18  Marketinq    Rasim  09:00:00  11:00:00
 4 2024-08-19         IT    Aynur  09:00:00  11:00:00)

In [ ]:
# Convert Entry and Exit columns to datetime to calculate working hours
attendance_data['Entry'] = pd.to_datetime(attendance_data['Entry'], format='%H:%M')
attendance_data['Exit'] = pd.to_datetime(attendance_data['Exit'], format='%H:%M')

# Calculate work hours per day for each employee
attendance_data['Work_Hours'] = (attendance_data['Exit'] - attendance_data['Entry']).dt.total_seconds() / 3600

# Now, we will calculate the total work hours, excluding permission times.
# First, convert Start and End of permission to datetime
permission_data['Start'] = pd.to_datetime(permission_data['Start'], format='%H:%M:%S')
permission_data['End'] = pd.to_datetime(permission_data['End'], format='%H:%M:%S')

# Calculate permission hours
permission_data['Permission_Hours'] = (permission_data['End'] - permission_data['Start']).dt.total_seconds() / 3600

# Merge attendance data with permission data on Date, Department, and Employee
attendance_with_permission = pd.merge(attendance_data, permission_data[['Date', 'Department', 'Employee', 'Permission_Hours']], 
                                      on=['Date', 'Department', 'Employee'], how='left')

# Subtract permission hours from work hours (if permission exists)
attendance_with_permission['Adjusted_Work_Hours'] = attendance_with_permission['Work_Hours'] - attendance_with_permission['Permission_Hours'].fillna(0)

# Display the adjusted work hours data
import ace_tools as tools; tools.display_dataframe_to_user(name="Adjusted Attendance with Permissions", dataframe=attendance_with_permission)
